In [52]:
import csv
import sys
import pickle
import pandas as pd
import numpy as np
from graphviz import Source
from sklearn import tree
from IPython.display import SVG
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from PIL import Image
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')

In [3]:
def read_dataset(tipo):
    data = pd.read_csv('/home/alison/Documentos/Projeto-Tomate/datasets_especies/dataset_especies_mfcc.csv', sep=',')

    if tipo == 'flor':
        data = data[data['Annotation'] != 'voo']
    elif tipo == 'voo':
        data = data[data['Annotation'] == 'voo']

    data = data.drop(['filename', 'Annotation'],axis=1)

    classes = data.iloc[:, -1]
    standard = StandardScaler()
    matriz = standard.fit_transform(np.array(data.iloc[:, :-1]))
    
    X_train, X_test, y_train, y_test = train_test_split(matriz, classes, test_size=0.5, random_state=0)

    return X_train, X_test, y_train, y_test

In [5]:
def treino_teste(X_train, y_train, modelo, tuned_parameters, filename):
    file_csv = filename + '.csv'
    file_pdf = filename + '.pdf'
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    score = 'f1_macro'

    clf = GridSearchCV(modelo, tuned_parameters, scoring=score, cv=kfold)

    clf.fit(X_train, y_train)

    #y_true = clf.predict(X_test)
    #if score == 'f1_macro':
    #    c, p = y_true, y_pred
    
    #print(metrics.classification_report(y_true, y_pred))
    #cm = pd.crosstab(c, p, rownames=['True'], colnames=['Predicted'], margins=True, dropna=False)
    #cm.to_csv(file_csv, sep='\t')

    #fig = plt.figure(num=None, figsize=(10, 10), dpi=50, facecolor='w', edgecolor='k')
    #tab_acertos = sns.heatmap(cm, cmap='PuRd', annot=True, annot_kws={'size':13}, cbar=False, square=True)
    #tab_acertos.set_xticklabels(tab_acertos.get_xticklabels(), rotation=90) 
    #tab_acertos.get_figure().savefig(file_pdf)
    #plt.show()
    
    return clf.best_params_

## Classificação com SVM

In [6]:
# CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

svc_model = SVC()
best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'espcieSVMambos')
print(best_params)

{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [8]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 56.5217
Precision..: 46.6608
Recall.....: 46.5979
F1-Score...: 45.1633

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         3
            Augochloropsis_sp1       0.58      0.64      0.61        11
            Augochloropsis_sp2       0.83      0.71      0.77         7
                  Bombus_morio       0.45      0.38      0.42        13
             Bombus_pauloensis       0.29      0.46      0.35        13
               Centris_tarsata       0.29      0.67      0.40         3
           Centris_trigonoides       0.67      0.57      0.62         7
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.74      0.76      0.75        41
             Exomalopsis_minor       1.00      0.64      0.78        11
              Melipona_bicolor       0.55      0.70      0.62        23
       Melipona_quadrifasciata       0.60      0.3

In [53]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'especieSVMflor')
print(best_params)

{'C': 10, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [10]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 73.3945
Precision..: 61.7508
Recall.....: 60.7020
F1-Score...: 59.0608

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       1.00      1.00      1.00         1
            Augochloropsis_sp1       0.67      0.33      0.44         6
            Augochloropsis_sp2       1.00      1.00      1.00         9
                  Bombus_morio       0.83      0.62      0.71         8
             Bombus_pauloensis       0.50      0.75      0.60         8
               Centris_tarsata       0.00      0.00      0.00         2
           Centris_trigonoides       1.00      1.00      1.00         2
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.78      0.93      0.85        30
             Exomalopsis_minor       1.00      0.64      0.78        11
              Melipona_bicolor       0.82      0.60      0.69        15
       Melipona_quadrifasciata       0.67      1.0

In [11]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['poly'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']},
                    {'kernel': ['sigmoid'], 'gamma': [1e-2,1e-3,1e-4], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}, 
                    {'kernel': ['linear'], 'C': [0.001, 0.1, 0.01, 1, 10], 'decision_function_shape': ['ovo']}]

best_params = treino_teste(X_train, y_train, svc_model, tuned_parameters, 'especieSVMvoo')
print(best_params)

{'C': 0.1, 'decision_function_shape': 'ovo', 'kernel': 'linear'}


In [13]:
svc_best_model = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')

svc_best_model.fit(X_train, y_train)

y_pred = svc_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 55.7692
Precision..: 56.9109
Recall.....: 53.7821
F1-Score...: 49.0065

                                precision    recall  f1-score   support

            Augochloropsis_sp1       1.00      0.38      0.55         8
            Augochloropsis_sp2       0.00      0.00      0.00         0
                  Bombus_morio       1.00      0.80      0.89         5
             Bombus_pauloensis       0.56      1.00      0.71         5
               Centris_tarsata       0.20      1.00      0.33         1
           Centris_trigonoides       1.00      0.50      0.67         6
            Exomalopsis_analis       0.14      0.50      0.22         4
             Exomalopsis_minor       0.00      0.00      0.00         4
              Melipona_bicolor       1.00      0.75      0.86         8
       Melipona_quadrifasciata       0.50      0.67      0.57         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       1.00      0.4

## Classificação LR

In [14]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRambos')
print(best_params)

{'C': 10}


In [15]:
lr_best_model = LogisticRegression(C=10)

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 53.4161
Precision..: 53.6747
Recall.....: 51.3621
F1-Score...: 48.6180

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         3
            Augochloropsis_sp1       0.46      0.55      0.50        11
            Augochloropsis_sp2       1.00      0.43      0.60         7
                  Bombus_morio       0.40      0.31      0.35        13
             Bombus_pauloensis       0.35      0.54      0.42        13
               Centris_tarsata       0.29      0.67      0.40         3
           Centris_trigonoides       0.75      0.43      0.55         7
               Eulaema_nigrita       0.50      1.00      0.67         1
            Exomalopsis_analis       0.72      0.76      0.74        41
             Exomalopsis_minor       0.83      0.45      0.59        11
              Melipona_bicolor       0.48      0.48      0.48        23
       Melipona_quadrifasciata       0.57      0.4

In [17]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRflor')
print(best_params)

{'C': 10}


In [18]:
lr_best_model = LogisticRegression(C=10)

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 64.2202
Precision..: 45.5688
Recall.....: 41.1330
F1-Score...: 41.2729

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         1
            Augochloropsis_sp1       0.50      0.33      0.40         6
            Augochloropsis_sp2       1.00      0.89      0.94         9
                  Bombus_morio       0.57      0.50      0.53         8
             Bombus_pauloensis       0.58      0.88      0.70         8
               Centris_tarsata       0.00      0.00      0.00         2
           Centris_trigonoides       1.00      0.50      0.67         2
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.75      0.90      0.82        30
             Exomalopsis_minor       0.75      0.55      0.63        11
              Melipona_bicolor       0.67      0.40      0.50        15
       Melipona_quadrifasciata       0.12      0.5

In [19]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = [{'penalty': ['l1', 'l2']},
                    {'C': [0.001, 0.1, 1, 10, 100]}]

lr_model = LogisticRegression()
best_params = treino_teste(X_train, y_train, lr_model, tuned_parameters, 'especieLRvoo')
print(best_params)

{'C': 10}


In [20]:
lr_best_model = LogisticRegression(C=10)

lr_best_model.fit(X_train, y_train)

y_pred = lr_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 51.9231
Precision..: 47.2534
Recall.....: 41.3095
F1-Score...: 40.2016

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       1.00      0.12      0.22         8
                  Bombus_morio       1.00      0.80      0.89         5
             Bombus_pauloensis       0.62      1.00      0.77         5
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       0.86      1.00      0.92         6
               Eulaema_nigrita       0.00      0.00      0.00         0
            Exomalopsis_analis       0.13      0.50      0.21         4
             Exomalopsis_minor       0.00      0.00      0.00         4
              Melipona_bicolor       0.83      0.62      0.71         8
       Melipona_quadrifasciata       0.50      0.33      0.40         3
Pseudoalglochloropsis_graminea       0.00      0.0

## Classificação RF

In [21]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFambos')
print(best_param)

{'max_features': 'auto', 'n_estimators': 200}


In [22]:
rf_best_model = RandomForestClassifier(max_features='auto', n_estimators=200)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 50.9317
Precision..: 40.0143
Recall.....: 38.5212
F1-Score...: 36.2172

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         3
            Augochloropsis_sp1       0.50      0.18      0.27        11
            Augochloropsis_sp2       1.00      0.71      0.83         7
                  Bombus_morio       0.67      0.62      0.64        13
             Bombus_pauloensis       0.27      0.46      0.34        13
               Centris_tarsata       0.40      0.67      0.50         3
           Centris_trigonoides       0.80      0.57      0.67         7
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.65      0.80      0.72        41
             Exomalopsis_minor       1.00      0.36      0.53        11
              Melipona_bicolor       0.38      0.57      0.46        23
       Melipona_quadrifasciata       0.00      0.0

In [23]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFflor')
print(best_param)

{'max_features': 'sqrt', 'n_estimators': 200}


In [24]:
rf_best_model = RandomForestClassifier(max_features='sqrt', n_estimators=200)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 61.4679
Precision..: 50.9583
Recall.....: 39.5875
F1-Score...: 40.1111

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         1
            Augochloropsis_sp1       1.00      0.33      0.50         6
            Augochloropsis_sp2       1.00      0.22      0.36         9
                  Bombus_morio       1.00      0.50      0.67         8
             Bombus_pauloensis       0.45      0.62      0.53         8
               Centris_tarsata       0.00      0.00      0.00         2
           Centris_trigonoides       0.00      0.00      0.00         2
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.56      0.93      0.70        30
             Exomalopsis_minor       1.00      0.55      0.71        11
              Melipona_bicolor       0.65      0.73      0.69        15
       Melipona_quadrifasciata       1.00      1.0

In [25]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {'n_estimators': [100, 200],
                    'max_features': ['auto', 'sqrt', 'log2']}

rf_model = RandomForestClassifier()
best_param = treino_teste(X_train, y_train, rf_model, tuned_parameters, 'especieRFvoo')
print(best_param)

{'max_features': 'auto', 'n_estimators': 100}


In [28]:
rf_best_model = RandomForestClassifier(max_features='auto', n_estimators=100)

rf_best_model.fit(X_train, y_train)

y_pred = rf_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 48.0769
Precision..: 45.1705
Recall.....: 48.1944
F1-Score...: 39.4298

                                precision    recall  f1-score   support

            Augochloropsis_sp1       0.00      0.00      0.00         8
                  Bombus_morio       1.00      1.00      1.00         5
             Bombus_pauloensis       0.50      1.00      0.67         5
               Centris_tarsata       0.25      1.00      0.40         1
           Centris_trigonoides       1.00      0.33      0.50         6
            Exomalopsis_analis       0.12      0.50      0.20         4
             Exomalopsis_minor       0.00      0.00      0.00         4
              Melipona_bicolor       0.55      0.75      0.63         8
       Melipona_quadrifasciata       1.00      1.00      1.00         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       1.00      0.20      0.33         5
             Xylocopa_suspecta       0.00      0.0

## Classificação DTree

In [29]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEambos')
print(best_param)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 2}


In [31]:
dtree_best_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=2)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 32.9193
Precision..: 31.6073
Recall.....: 28.5408
F1-Score...: 25.6790

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         3
            Augochloropsis_sp1       0.57      0.36      0.44        11
            Augochloropsis_sp2       0.00      0.00      0.00         7
                  Bombus_morio       0.21      0.23      0.22        13
             Bombus_pauloensis       0.38      0.38      0.38        13
               Centris_tarsata       0.40      0.67      0.50         3
           Centris_trigonoides       0.43      0.43      0.43         7
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.46      0.41      0.44        41
             Exomalopsis_minor       0.50      0.45      0.48        11
              Melipona_bicolor       0.44      0.30      0.36        23
       Melipona_quadrifasciata       1.00      0.1

In [32]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEflor')
print(best_param)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 10}


In [34]:
dtree_best_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=2)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 43.1193
Precision..: 32.8175
Recall.....: 37.5993
F1-Score...: 31.6904

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         1
            Augochloropsis_sp1       0.20      0.17      0.18         6
            Augochloropsis_sp2       0.67      0.44      0.53         9
                  Bombus_morio       0.00      0.00      0.00         8
             Bombus_pauloensis       0.25      0.25      0.25         8
               Centris_tarsata       0.00      0.00      0.00         2
           Centris_trigonoides       0.50      0.50      0.50         2
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.50      0.70      0.58        30
             Exomalopsis_minor       0.71      0.45      0.56        11
              Melipona_bicolor       0.80      0.53      0.64        15
       Melipona_quadrifasciata       0.33      1.0

In [35]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {"criterion": ["gini", "entropy"],
                    "min_samples_split": [2, 10],
                    "max_depth": [2, 5, 10]
                    }

detree_model = DecisionTreeClassifier()
best_param = treino_teste(X_train, y_train, detree_model, tuned_parameters, 'especieDTREEvoo')
print(best_param)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 10}


In [38]:
dtree_best_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=2)

dtree_best_model.fit(X_train, y_train)

y_pred = dtree_best_model.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 48.0769
Precision..: 34.3537
Recall.....: 32.2024
F1-Score...: 32.0292

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         0
            Augochloropsis_sp1       0.67      0.50      0.57         8
            Augochloropsis_sp2       0.00      0.00      0.00         0
                  Bombus_morio       0.50      0.40      0.44         5
             Bombus_pauloensis       0.67      0.80      0.73         5
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       1.00      0.67      0.80         6
            Exomalopsis_analis       0.14      0.25      0.18         4
             Exomalopsis_minor       0.00      0.00      0.00         4
              Melipona_bicolor       1.00      0.62      0.77         8
       Melipona_quadrifasciata       0.33      0.67      0.44         3
Pseudoalglochloropsis_graminea       0.00      0.0

## Classificação Ensemble

In [40]:
#CLASSIFICAÇÃO SONICATION + FLIGHT
X_train, X_test, y_train, y_test = read_dataset('ambos')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEambos')
print(best_param)

{'lr__C': 100, 'svc__C': 10}


In [46]:
best_svc = SVC(C=10)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=10)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 58.3851
Precision..: 55.2265
Recall.....: 47.1490
F1-Score...: 47.9372

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         3
            Augochloropsis_sp1       0.60      0.55      0.57        11
            Augochloropsis_sp2       1.00      0.57      0.73         7
                  Bombus_morio       0.78      0.54      0.64        13
             Bombus_pauloensis       0.27      0.46      0.34        13
               Centris_tarsata       0.29      0.67      0.40         3
           Centris_trigonoides       0.60      0.43      0.50         7
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.71      0.83      0.76        41
             Exomalopsis_minor       1.00      0.55      0.71        11
              Melipona_bicolor       0.48      0.65      0.56        23
       Melipona_quadrifasciata       0.67      0.4

In [47]:
#CLASSIFICAÇÃO SONICATION
X_train, X_test, y_train, y_test = read_dataset('flor')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEflor')
print(best_param)

{'lr__C': 10, 'svc__C': 10}


In [48]:
best_svc = SVC(C=10)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=10)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 65.1376
Precision..: 43.7991
Recall.....: 37.8737
F1-Score...: 38.8633

                                precision    recall  f1-score   support

  Augochloropsis_brachycephala       0.00      0.00      0.00         1
            Augochloropsis_sp1       0.40      0.33      0.36         6
            Augochloropsis_sp2       1.00      0.67      0.80         9
                  Bombus_morio       1.00      0.50      0.67         8
             Bombus_pauloensis       0.44      0.88      0.58         8
               Centris_tarsata       0.00      0.00      0.00         2
           Centris_trigonoides       0.00      0.00      0.00         2
               Eulaema_nigrita       0.00      0.00      0.00         1
            Exomalopsis_analis       0.65      0.93      0.77        30
             Exomalopsis_minor       1.00      0.55      0.71        11
              Melipona_bicolor       0.69      0.60      0.64        15
       Melipona_quadrifasciata       0.50      0.5

In [49]:
#CLASSIFICAÇÃO FLIGHT
X_train, X_test, y_train, y_test = read_dataset('voo')

tuned_parameters = {'lr__C': [0.001, 0.1, 1, 10, 100],
                    'svc__C': [0.001, 0.1, 0.01, 1, 10]}

svc = SVC()
rf = RandomForestClassifier()
lr = LogisticRegression()

modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

votingclf = VotingClassifier(estimators=modelos, voting='hard')
best_param = treino_teste(X_train, y_train, votingclf, tuned_parameters, 'especieENSEMBLEvoo')
print(best_param)

{'lr__C': 10, 'svc__C': 1}


In [51]:
best_svc = SVC(C=1)
rf = RandomForestClassifier()
best_lr = LogisticRegression(C=10)

modelos = [('svc', best_svc), ('rf', rf), ('lr', best_lr)]

best_votingclf = VotingClassifier(estimators=modelos, voting='hard')

best_votingclf.fit(X_train, y_train)

y_pred = best_votingclf.predict(X_test)

print("Acurácia...: %.4f" %(metrics.accuracy_score(y_test, y_pred) * 100))
print("Precision..: %.4f" %(metrics.precision_score(y_test, y_pred, average='macro') * 100))
print("Recall.....: %.4f" %(metrics.recall_score(y_test, y_pred, average='macro') * 100))
print("F1-Score...: %.4f" %(metrics.f1_score(y_test, y_pred, average='macro') * 100))
print()
print(metrics.classification_report(y_test, y_pred))
print()

Acurácia...: 50.0000
Precision..: 49.6528
Recall.....: 40.9028
F1-Score...: 38.2898

                                precision    recall  f1-score   support

            Augochloropsis_sp1       1.00      0.12      0.22         8
                  Bombus_morio       1.00      1.00      1.00         5
             Bombus_pauloensis       0.50      1.00      0.67         5
               Centris_tarsata       0.00      0.00      0.00         1
           Centris_trigonoides       1.00      0.67      0.80         6
            Exomalopsis_analis       0.12      0.50      0.20         4
             Exomalopsis_minor       0.00      0.00      0.00         4
              Melipona_bicolor       0.67      0.75      0.71         8
       Melipona_quadrifasciata       0.67      0.67      0.67         3
Pseudoalglochloropsis_graminea       0.00      0.00      0.00         2
          Xylocopa_nigrocincta       1.00      0.20      0.33         5
             Xylocopa_suspecta       0.00      0.0